In [1]:
import transformers
import json

import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets

import pandas as pd
import json
import numpy as np
covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'

def get_data_from_json(filename):
    jsonfile = open(covid_file, 'r')
    data = jsonfile.read()
    jsonfile.close()
    return json.loads(data)

#datasets.set_caching_enabled(False)

Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


In [2]:
from collections import OrderedDict
def make_and_save_full_dataset(covid=None, squad = None, bioASQ = None, path = '../data/squad_bioASQ_covidQA/'):
    squad = datasets.Dataset.from_dict(squad_qa[:])
    bioASQ = datasets.Dataset.from_dict(bioASQ[:])
    if covid is not None:
        full_data = datasets.dataset_dict.DatasetDict({'squad':squad, 'covid':covid,  'bio':bioASQ})
    else:
        full_data = datasets.dataset_dict.DatasetDict({'squad':squad,'bio':bioASQ})

    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

In [3]:
covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)
squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])

covid_bio_squad_dataset_path = "../data/squad_bioASQ_covidQA_method_one"

#this is just for testing purposes, I am going to make both of these files very small only at max 3000 datasets
squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
bio_qa = datasets.Dataset.from_dict(bio_qa[:20])
covid_qa = datasets.Dataset.from_dict(covid_qa[:10])

Using custom data configuration default-a396986d6b3a2375
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-a396986d6b3a2375\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-8fdbe041288a2f4d
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-8fdbe041288a2f4d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [4]:
make_and_save_full_dataset(squad=squad_qa,bioASQ=bio_qa,path=covid_bio_squad_dataset_path)

K = 4
to_remove_per_step = int(squad_qa.num_rows / K)
bio_remove_per_step = int(bio_qa.num_rows / K)

In [5]:
def run_gradual_ft(output_dir, checkpoint, k_fold, dataset_path):
    !python run_qa.py \
      --model_name_or_path {checkpoint} \
      --dataset_name {dataset_path}\
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 32\
      --per_device_eval_batch_size 32\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 1e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --k_fold_cross_valid {k_fold} \
      --output_dir {output_dir} \
      --overwrite_output_dir


In [6]:
k_fold = 5
for i in range(K):
    directory = '../models/gradual_ft_baseline_method_one/checkpoint-' + str(i+1) + '/'
    output_dir = '../models/gradual_ft_baseline_method_one/checkpoint-' + str(i) + '/' + 'Split-'
    print('\n\n**************************************************')
    print('==================================================')
    print('          At Gradual Fine Tuning Step: ',i+1)
    print('==================================================')
    print('**************************************************\n\n')
    if i < 1:
        run_gradual_ft(directory,'roberta-base', k_fold, covid_bio_squad_dataset_path)
    else:
        run_gradual_ft(directory, output_dir, k_fold, covid_bio_squad_dataset_path)
        squad_qa.shuffle()
        bio_qa.shuffle()
        covid_qa = datasets.Dataset.from_dict(covid_qa[:])

    if i%2:
        if to_remove_per_step > squad_qa.num_rows:
            print('not enough data')
            break
        squad_qa = datasets.Dataset.from_dict(squad_qa[:-to_remove_per_step])
        make_and_save_full_dataset(squad=squad_qa, bioASQ= bio_qa, path=covid_bio_squad_dataset_path)
    else:
        if to_remove_per_step > bio_qa.num_rows:
            print('not enough data')
            break
        bio_qa = datasets.Dataset.from_dict(bio_qa[:-bio_remove_per_step])
        make_and_save_full_dataset(squad=squad_qa, bioASQ=bio_qa,path=covid_bio_squad_dataset_path)

print('Finished process')



**************************************************
          At Gradual Fine Tuning Step:  1
**************************************************


07/01/2021 11:30:34 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/01/2021 11:30:34 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
leng

[INFO|configuration_utils.py:530] 2021-07-01 11:30:34,947 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:566] 2021-07-01 11:30:34,947 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "u

***** predict metrics *****
  predict_samples  =      32
  test_exact_match =     0.0
  test_f1          = 12.9032


  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_auto.py:427] 2021-07-01 11:31:02,396 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:530] 2021-07-01 11:31:02,715 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:566] 2021-07-01 11:31:02,716 >> Mo

07/01/2021 11:31:37 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/01/2021 11:31:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|configuration_utils.py:528] 2021-07-01 11:31:37,786 >> loading configuration file ../models/gradual_ft_baseline_method_one/checkpoint-1/Split-5/config.json
[INFO|configuration_utils.py:566] 2021-07-01 11:31:37,786 >> Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_utils_base.py:1651] 2021-07-01 11:31:37,789 >> Didn't find file ../models

07/01/2021 11:32:21 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/01/2021 11:32:21 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|configuration_utils.py:528] 2021-07-01 11:32:22,384 >> loading configuration file ../models/gradual_ft_baseline_method_one/checkpoint-2/Split-5/config.json
[INFO|configuration_utils.py:566] 2021-07-01 11:32:22,384 >> Model config RobertaConfig {
  "_name_or_path": "../models/gradual_ft_baseline_method_one/checkpoint-1/Split-4/",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_utils_base.py:1651] 202

07/01/2021 11:33:06 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/01/2021 11:33:06 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|configuration_utils.py:528] 2021-07-01 11:33:06,740 >> loading configuration file ../models/gradual_ft_baseline_method_one/checkpoint-3/Split-5/config.json
[INFO|configuration_utils.py:566] 2021-07-01 11:33:06,740 >> Model config RobertaConfig {
  "_name_or_path": "../models/gradual_ft_baseline_method_one/checkpoint-2/Split-4/",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_utils_base.py:1651] 202